In [2]:
# Installs geemap package
import subprocess

try:
    import geemap
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

# Checks whether this notebook is running on Google Colab
try:
    import google.colab
    import geemap.eefolium as emap
except:
    import geemap as emap

# Authenticates and initializes Earth Engine
import ee

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()  

geemap package not installed. Installing ...
To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=hljlUSrr4B1q7Hozr31x465bjO07bmzDC676YC5ctE8&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWhxq9ZhCke2OI8tJiBMw_MBrLQQyr_BepIMeatrZOsO8UuW3cg3chY

Successfully saved authorization token.


In [ ]:

# Algorithm:
# 1. Get counties of interest in Snake River Plain (see table below)
# 2. Create a new feature with selected counties
# 3. Union the counties, export the counties
# 4. Get the NLCD data
# 5. Clip to the area of the county
# 6. Export 

#--------------------------------#
#   County         |  FIPS code  #
#--------------------------------#
#   Ada            |   16001     #
#   Boise          |   16015     #
#   Canyon         |   16027     #
#   Elmore         |   16039     #
#   Gem            |   16045     #
#--------------------------------#




In [3]:
counties = ee.FeatureCollection("TIGER/2016/Counties")


lbrb_countyfp_list = ee.List(['001','015','027','039','045'])

id_lbrb_cnty = counties.filter(ee.Filter.eq('STATEFP','16')) \
                      .filter(ee.Filter.inList('COUNTYFP', lbrb_countyfp_list)) \

lbrb_bnd = id_lbrb_cnty.union()

# Create a new map and add the watershed we will use to clip the EVI data
Map = emap.Map(center=(43.6150, -116.2023),zoom=6)

Map.addLayer(ee.Image().paint(lbrb_bnd, 0, 2), {}, 'Lower Boise River Basin')
Map.addLayerControl()
Map

In [6]:
dataset = ee.ImageCollection('USDA/NASS/CDL').filterBounds(lbrb_bnd)
lbrb_cropland = dataset.select('cropland') \
                    .map(lambda image: image.clip(lbrb_bnd))
croplandVis = {
  'min': 1.0,
  'max': 254.0,
  'palette': ['ffd300', #corn, dbl crop winwheat/corn, oats/corn, barley/corn
              #corn/soybeans
              'ff2626', #cotton, lettuce/cotton
              '00a832', #rice
              'ff9e0a', #sorghum, durum wheat/sorghum, barley/sorghum
              '267000', #soybeans, soybeans/cotton, soybeans/oats, barley/soybeans
              'ffff00', #sunflower
              '70a500', #peanuts
              '00af49', #tobacco, mustard, switchgrass
              'dda50a', #sweet corn
              '7cd3ff', #mint, herbs
              'e2007c', #Barley, lettuce/barley
              '896054', #durum wheat
              'd8b56b', #spring wheat
              'a57000', #winter wheat, winwheat/sorghum, winwheat/cotton
              'd69ebc', #other small grains, speltz, buckwheat, triticale
              '707000', #Dbl crop winwht/soybeans
              'aa007c', #rye
              'a05989',  #oats
              '700049', #millet
              'd1ff00', #canola, rape seed
              '7c99ff', #flaxseed
              'd6d600', #safflower
              'ffa5e2', #alfalfa
              'a5f28c', #other hay/non alfalfa
              '00af49', #camelina, other crops, chickpeas, hops, vetch
              'a800e2', #sugarbeets
              'a50000', #dry beans
              '702600', #potatoes, sweet potatoes
              'af7cff', #sugarcane
              'ff6666', #misc vegs and fruits, cucumbers, caneberries, watermelons
              #carrots, asparagus, garlic, cantelope, honeydew melons, broccoli,
              #peppers, greens, strawberries, squash, lettuce, pumpkins, cabbage
              #cauliflower, celery, radish, turnips, eggplant, gourds, cranberries
              'ffcc66', #onions
              '00ddaf', #lentils
              '54ff00', #peas
              'f2a377', #tomatoes
              'e8bfff', #clover/wildflowers
              'afffdd', #sod, grass seed
              'bfbf77', #fallow/ idle cropland
              '93cc93', #forest (deciduous, evergreen, mixed)
              'c6d69e', #shrubland
              'ccbfa3', #barren
              'ff00ff', #cherries
              'ff8eaa', #peaches, prunes, plums, nectarines, apricots
              'ba004f', #apples
              '704489', #grapes
              '007777', #Christmas trees
              'af9970', #other tree crops, pears, pomegranates
              'ffff7c', #citrus
              'b5705b', #pecans
              '00a582', #almonds
              'e8d6af', #walnuts
              'f2f2f2', #clouds/no data
              '999999', #developed, developed/open space, developed/low intensity
              #developed/med intensity, developed/high intensity
              '4970a3', #water, open water
              '7cafaf', #wetlands (woody and herbaceous)
              'e8ffbf', #nonag/undefined, grassland/pasture
              '00ffff', #aquaculture
              'd3e2f9', #perenial ice/snow
              '00ff8c', #pistachios
              '334933', #olives
              'e27026', #oranges
              '896054', #dbl crop lettuce/durum wheat
              '000099', #blueberries
  ]}

Map = emap.Map(center=(43.6150, -116.2023),zoom=8)
Map.addLayer(lbrb_cropland, croplandVis, 'Cropland')
Map


In [ ]:
dataset = ee.ImageCollection("USDA/NASS/CDL": {
        "1 Corn": "ffd300",
        "2 Cotton": "ff2626",
        "3 Rice": "00a8e2",
        "4 Sorghum": "ff9e0a",
        "5 Soybeans": "267000",
        "6 Sunflower": "ffff00",
        "10 Peanuts": "70a500",
        "11 Tobacco": "00af49",
        "12 Sweet Corn": "dda50a",
        "13 Pop or Orn Corn": "dda50a",
        "14 Mint": "7cd3ff",
        "21 Barley": "e2007c",
        "22 Durum Wheat": "896054",
        "23 Spring Wheat": "d8b56b",
        "24 Winter Wheat": "a57000",
        "25 Other Small Grains": "d69ebc",
        "26 Dbl Crop WinWht/Soybeans": "707000",
        "27 Rye": "aa007c",
        "28 Oats": "a05989",
        "29 Millet": "700049",
        "30 Speltz": "d69ebc",
        "31 Canola": "d1ff00",
        "32 Flaxseed": "7c99ff",
        "33 Safflower": "d6d600",
        "34 Rape Seed": "d1ff00",
        "35 Mustard": "00af49",
        "36 Alfalfa": "ffa5e2",
        "37 Other Hay/Non Alfalfa": "a5f28c",
        "38 Camelina": "00af49",
        "39 Buckwheat": "d69ebc",
        "41 Sugarbeets": "a800e2",
        "42 Dry Beans": "a50000",
        "43 Potatoes": "702600",
        "44 Other Crops": "00af49",
        "45 Sugarcane": "af7cff",
        "46 Sweet Potatoes": "702600",
        "47 Misc Vegs & Fruits": "ff6666",
        "48 Watermelons": "ff6666",
        "49 Onions": "ffcc66",
        "50 Cucumbers": "ff6666",
        "51 Chick Peas": "00af49",
        "52 Lentils": "00ddaf",
        "53 Peas": "54ff00",
        "54 Tomatoes": "f2a377",
        "55 Caneberries": "ff6666",
        "56 Hops": "00af49",
        "57 Herbs": "7cd3ff",
        "58 Clover/Wildflowers": "e8bfff",
        "59 Sod/Grass Seed": "afffdd",
        "60 Switchgrass": "00af49",
        "61 Fallow/Idle Cropland": "bfbf77",
        "63 Forest": "93cc93",
        "64 Shrubland": "c6d69e",
        "65 Barren": "ccbfa3",
        "66 Cherries": "ff00ff",
        "67 Peaches": "ff8eaa",
        "68 Apples": "ba004f",
        "69 Grapes": "704489",
        "70 Christmas Trees": "007777",
        "71 Other Tree Crops": "af9970",
        "72 Citrus": "ffff7c",
        "74 Pecans": "b5705b",
        "75 Almonds": "00a582",
        "76 Walnuts": "e8d6af",
        "77 Pears": "af9970",
        "81 Clouds/No Data": "f2f2f2",
        "82 Developed": "999999",
        "83 Water": "4970a3",
        "87 Wetlands": "7cafaf",
        "88 Nonag/Undefined": "e8ffbf",
        "92 Aquaculture": "00ffff",
        "111 Open Water": "4970a3",
        "112 Perennial Ice/Snow": "d3e2f9",
        "121 Developed/Open Space": "999999",
        "122 Developed/Low Intensity": "999999",
        "123 Developed/Med Intensity": "999999",
        "124 Developed/High Intensity": "999999",
        "131 Barren": "ccbfa3",
        "141 Deciduous Forest": "93cc93",
        "142 Evergreen Forest": "93cc93",
        "143 Mixed Forest": "93cc93",
        "152 Shrubland": "c6d69e",
        "176 Grassland/Pasture": "e8ffbf",
        "190 Woody Wetlands": "7cafaf",
        "195 Herbaceous Wetlands": "7cafaf",
        "204 Pistachios": "00ff8c",
        "205 Triticale": "d69ebc",
        "206 Carrots": "ff6666",
        "207 Asparagus": "ff6666",
        "208 Garlic": "ff6666",
        "209 Cantaloupes": "ff6666",
        "210 Prunes": "ff8eaa",
        "211 Olives": "334933",
        "212 Oranges": "e27026",
        "213 Honeydew Melons": "ff6666",
        "214 Broccoli": "ff6666",
        "216 Peppers": "ff6666",
        "217 Pomegranates": "af9970",
        "218 Nectarines": "ff8eaa",
        "219 Greens": "ff6666",
        "220 Plums": "ff8eaa",
        "221 Strawberries": "ff6666",
        "222 Squash": "ff6666",
        "223 Apricots": "ff8eaa",
        "224 Vetch": "00af49",
        "225 Dbl Crop WinWht/Corn": "ffd300",
        "226 Dbl Crop Oats/Corn": "ffd300",
        "227 Lettuce": "ff6666",
        "229 Pumpkins": "ff6666",
        "230 Dbl Crop Lettuce/Durum Wht": "896054",
        "231 Dbl Crop Lettuce/Cantaloupe": "ff6666",
        "232 Dbl Crop Lettuce/Cotton": "ff2626",
        "233 Dbl Crop Lettuce/Barley": "e2007c",
        "234 Dbl Crop Durum Wht/Sorghum": "ff9e0a",
        "235 Dbl Crop Barley/Sorghum": "ff9e0a",
        "236 Dbl Crop WinWht/Sorghum": "a57000",
        "237 Dbl Crop Barley/Corn": "ffd300",
        "238 Dbl Crop WinWht/Cotton": "a57000",
        "239 Dbl Crop Soybeans/Cotton": "267000",
        "240 Dbl Crop Soybeans/Oats": "267000",
        "241 Dbl Crop Corn/Soybeans": "ffd300",
        "242 Blueberries": "000099",
        "243 Cabbage": "ff6666",
        "244 Cauliflower": "ff6666",
        "245 Celery": "ff6666",
        "246 Radishes": "ff6666",
        "247 Turnips": "ff6666",
        "248 Eggplants": "ff6666",
        "249 Gourds": "ff6666",
        "250 Cranberries": "ff6666",
        "254 Dbl Crop Barley/Soybeans": "267000",
    }.filterBounds(lbrb_srb)

SyntaxError: ignored

In [ ]:
downConfig = {'scale': 30, "maxPixels": 1.0E13, 'driveFolder': 'srp-nlcd'}  # scale means resolution.
img_lst = srb_landcover.toList(100)

count = srb_landcover.size().getInfo()

for i in range(0, count):
    image = ee.Image(img_lst.get(i))
    name = image.get('system:index').getInfo()
    print(name)
    task = ee.batch.Export.image(image.clip(srb_bnd.geometry()), name, downConfig)
    task.start()
    print(task.status())

NLCD1992
{'state': 'READY', 'description': 'NLCD1992', 'creation_timestamp_ms': 1602450996161, 'update_timestamp_ms': 1602450996161, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'D35E3TPVLPHPIXST7VZDLMRD', 'name': 'projects/earthengine-legacy/operations/D35E3TPVLPHPIXST7VZDLMRD'}
NLCD2001
{'state': 'READY', 'description': 'NLCD2001', 'creation_timestamp_ms': 1602451018960, 'update_timestamp_ms': 1602451018960, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'UNT7DBP4SQRBURSUVD466POV', 'name': 'projects/earthengine-legacy/operations/UNT7DBP4SQRBURSUVD466POV'}
NLCD2004
{'state': 'READY', 'description': 'NLCD2004', 'creation_timestamp_ms': 1602451040720, 'update_timestamp_ms': 1602451040720, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'MXGCVL6NHIIBKYX62RCAY43D', 'name': 'projects/earthengine-legacy/operations/MXGCVL6NHIIBKYX62RCAY43D'}
NLCD2006
{'state': 'READY', 'description': 'NLCD2006', 'creation_timestamp_ms': 1602451059662, 'update_timestamp

In [ ]:
task.status()

{'creation_timestamp_ms': 1602451143507,
 'description': 'NLCD2016',
 'id': 'VHMYVOWJI5VC7UR4CGUYWXAS',
 'name': 'projects/earthengine-legacy/operations/VHMYVOWJI5VC7UR4CGUYWXAS',
 'start_timestamp_ms': 0,
 'state': 'READY',
 'task_type': 'EXPORT_IMAGE',
 'update_timestamp_ms': 1602451143507}